In [44]:
import pandas as pd
import numpy as np

In [3]:
df = pd.read_csv('/Users/serdarevichar/Library/CloudStorage/GoogleDrive-serdarevichar@gmail.com/My Drive/fantasy-football-database.csv', index_col = 'Unnamed: 0')

In [4]:
week = df.loc[(df['Year'] == 2019) & (df['Week'] == 1)]
week

,Year,Week,Playoff Flag,Home Team,Home Score,Home Projection,Away Team,Away Score,Away Projection
0,2019,1,False,Justin,158.02,115.55,The,113.20,121.01
1,2019,1,False,Dante,107.92,118.65,Zach,126.14,114.46
2,2019,1,False,Michael,112.68,118.23,McGwire,134.32,122.20
3,2019,1,False,Andrew,128.54,121.59,Carter,141.26,117.11
4,2019,1,False,Haris,135.80,119.44,Ethan,101.82,112.52
5,2019,1,False,Noah,153.86,125.17,Nathan,132.22,116.77


In [7]:
scoreboard = week[['Home Team','Home Score','Away Score','Away Team']].copy()
scoreboard

,Home Team,Home Score,Away Score,Away Team
0,Justin,158.02,113.20,The
1,Dante,107.92,126.14,Zach
2,Michael,112.68,134.32,McGwire
3,Andrew,128.54,141.26,Carter
4,Haris,135.80,101.82,Ethan
5,Noah,153.86,132.22,Nathan


In [36]:
temp1 = scoreboard[['Home Team','Home Score','Away Score']].copy()
temp2 = scoreboard[['Away Team','Away Score','Home Score']].copy()

temp1.rename(columns={'Home Team':'Team', 'Home Score':'Score', 'Away Score':'Opp Score'}, inplace=True)
temp2.rename(columns={'Away Team':'Team', 'Away Score':'Score', 'Home Score':'Opp Score'}, inplace=True)

luck_data = pd.concat([temp1,temp2], axis=0, ignore_index=True)
luck_data

,Team,Score,Opp Score
0,Justin,158.02,113.20
1,Dante,107.92,126.14
2,Michael,112.68,134.32
3,Andrew,128.54,141.26
4,Haris,135.80,101.82
5,Noah,153.86,132.22
6,The,113.20,158.02
7,Zach,126.14,107.92
8,McGwire,134.32,112.68
9,Carter,141.26,128.54


In [41]:
luck_data['Win'] = luck_data['Score'] > luck_data['Opp Score']
luck_data['Win'] = luck_data['Win'].map(int)
luck_data

,Team,Score,Opp Score,Win
0,Justin,158.02,113.20,1
1,Dante,107.92,126.14,0
2,Michael,112.68,134.32,0
3,Andrew,128.54,141.26,0
4,Haris,135.80,101.82,1
5,Noah,153.86,132.22,1
6,The,113.20,158.02,0
7,Zach,126.14,107.92,1
8,McGwire,134.32,112.68,1
9,Carter,141.26,128.54,1


In [61]:
mean = luck_data['Score'].mean()
std = luck_data['Score'].std()

print(mean,std)

128.81499999999997 17.587392436835906


In [76]:
def opp_luck_score(score, mean=None, std=None):
    return int(((mean - score) / np.absolute(mean - score)) * np.floor(np.absolute(mean - score) / std))

In [77]:
opp_luck_score(100,mean=mean,std=std)

1

In [78]:
luck_data['Opp Luck Score'] = luck_data[['Opp Score']].map(opp_luck_score, mean=mean, std=std)
luck_data

,Team,Score,Opp Score,Win,Opp Luck Score
0,Justin,158.02,113.20,1,0
1,Dante,107.92,126.14,0,0
2,Michael,112.68,134.32,0,0
3,Andrew,128.54,141.26,0,0
4,Haris,135.80,101.82,1,1
5,Noah,153.86,132.22,1,0
6,The,113.20,158.02,0,-1
7,Zach,126.14,107.92,1,1
8,McGwire,134.32,112.68,1,0
9,Carter,141.26,128.54,1,0


In [79]:
def your_luck_score(score, opp_score):
    return np.floor(np.absolute(mean - score) / s) * (result * xi(2*mean - score) - (1-result) * xi(score))

In [80]:
your_luck_score(175,134)

NameError: name 'result' is not defined